In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

In [5]:
filepath = 'https://raw.githubusercontent.com/alfonso-santos/microcredencial-carteras-python-2023/main/Tema_5_APT/data/sp500_tickers.csv'
tickers_sp500 = list(pd.read_csv(filepath))
start_date = '2000-01-01'

precios = yf.download(tickers_sp500, start=start_date)['Adj Close']

precios_activos_sp500 = precios.copy()
precios_activos_sp500.dropna(axis=1, inplace=True)
ret_activos = np.log(precios_activos_sp500).diff().dropna()

[*********************100%%**********************]  503 of 503 completed

4 Failed downloads:
['BRK.B', 'PEAK', 'CDAY']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2000-01-01 -> 2024-05-20)')


In [37]:
# Función para calcular los retornos medios del siguiente período para los tercios superior e inferior
def calculate_terciles_means(df):
    
    top_tickers = []
    bottom_tickers = []
    # Desplazar el DataFrame para obtener los retornos del siguiente período
    df_next = df.shift(-1)

    # Ordenar los retornos del período actual
    sorted_returns = df.apply(lambda x: x.sort_values().values, axis=1)

    # Identificar el tamaño de cada tercio
    tercio_size = df.shape[1] // 3

    # Obtener los índices de los tercios superior e inferior
    top_indices = np.argsort(df.values, axis=1)[:, -tercio_size:]
    bottom_indices = np.argsort(df.values, axis=1)[:, :tercio_size]


    # Calcular los retornos medios del siguiente período para los activos identificados
    top_tercile_means = np.array([df_next.values[i, top_indices[i]].mean() for i in range(df.shape[0] - 1)])
    bottom_tercile_means = np.array([df_next.values[i, bottom_indices[i]].mean() for i in range(df.shape[0] - 1)])
    
    df_result = pd.DataFrame({
        'Top_Tercile_Mean_Next_Period': top_tercile_means,
        'Bottom_Tercile_Mean_Next_Period': bottom_tercile_means
    }, index=df.index[:-1])

    # Crear el DataFrame con los resultados
    return df_result
# Calcular los retornos medios del siguiente período para los tercios superior e inferior


In [38]:
terciles_means_next_period = calculate_terciles_means(ret_act_6m)

print(terciles_means_next_period)


            Top_Tercile_Mean_Next_Period  Bottom_Tercile_Mean_Next_Period
Date                                                                     
2000-07-31                      0.061798                         0.105478
2001-01-31                      0.037270                        -0.061724
2001-07-31                      0.060643                        -0.049834
2002-01-31                     -0.085752                        -0.316782
2002-07-31                     -0.034806                        -0.021222
2003-01-31                      0.218536                         0.252762
2003-07-31                      0.186781                         0.155056
2004-01-31                      0.033984                        -0.000364
2004-07-31                      0.140770                         0.092380
2005-01-31                      0.108004                         0.090764
2005-07-31                      0.119139                         0.047975
2006-01-31                      0.0063